In [1]:
# Imports necessários
import pandas as pd
import numpy as np
import json
import warnings
from pathlib import Path
from datetime import datetime
import ast

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliotecas importadas com sucesso!")
print(f"📅 Execução do ETL: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Bibliotecas importadas com sucesso!
📅 Execução do ETL: 2025-11-23 16:14:17


In [2]:
# Definir caminhos
base_path = Path(r'c:\Users\Lenovo\Documents\Pedro\Bancos2\projeto-bancos-dados-2')
raw_path = base_path / 'Data Layer' / 'raw' / 'dados_brutos'
silver_path = base_path / 'Data Layer' / 'silver' / 'dados_processados'

# Criar diretório silver se não existir
silver_path.mkdir(parents=True, exist_ok=True)

print(f"📂 Camada RAW: {raw_path}")
print(f"📂 Camada SILVER: {silver_path}")
print(f"✅ Diretório SILVER criado/verificado")

📂 Camada RAW: c:\Users\Lenovo\Documents\Pedro\Bancos2\projeto-bancos-dados-2\Data Layer\raw\dados_brutos
📂 Camada SILVER: c:\Users\Lenovo\Documents\Pedro\Bancos2\projeto-bancos-dados-2\Data Layer\silver\dados_processados
✅ Diretório SILVER criado/verificado


In [3]:
# Funções auxiliares para parsing JSON
def safe_json_loads(json_str, default=None):
    """Carrega JSON com tratamento de erros"""
    if pd.isna(json_str):
        return default
    try:
        # Tentar como JSON
        return json.loads(json_str)
    except:
        try:
            # Tentar como literal Python
            return ast.literal_eval(json_str)
        except:
            return default

def extract_names(json_str, key='name', max_items=None):
    """Extrai lista de nomes de um campo JSON"""
    data = safe_json_loads(json_str, [])
    if not isinstance(data, list):
        return []
    names = [item.get(key, '') for item in data if isinstance(item, dict)]
    return names[:max_items] if max_items else names

def extract_first_name(json_str, key='name', default=None):
    """Extrai o primeiro nome de um campo JSON"""
    names = extract_names(json_str, key, max_items=1)
    return names[0] if names else default

def extract_genres(json_str):
    """Extrai gêneros como string separada por pipe"""
    genres = extract_names(json_str, 'name')
    return '|'.join(genres) if genres else None

def extract_director(crew_json):
    """Extrai o nome do diretor do JSON de crew"""
    crew = safe_json_loads(crew_json, [])
    if not isinstance(crew, list):
        return None
    for person in crew:
        if isinstance(person, dict) and person.get('job') == 'Director':
            return person.get('name')
    return None

def extract_top_actors(cast_json, n=5):
    """Extrai os N principais atores do JSON de cast"""
    cast = safe_json_loads(cast_json, [])
    if not isinstance(cast, list):
        return None
    # Ordenar por 'order' (ordem de importância)
    sorted_cast = sorted(cast, key=lambda x: x.get('order', 999) if isinstance(x, dict) else 999)
    actors = [person.get('name', '') for person in sorted_cast[:n] if isinstance(person, dict)]
    return '|'.join(actors) if actors else None

print("✅ Funções auxiliares definidas!")

✅ Funções auxiliares definidas!


---
## 1️⃣ Transformação: Movies Metadata

In [4]:
# Carregar dados RAW
print("📥 Carregando movies_metadata.csv...")
movies_raw = pd.read_csv(raw_path / 'movies_metadata.csv', low_memory=False)
print(f"✅ {len(movies_raw):,} filmes carregados")
print(f"📊 Dimensões originais: {movies_raw.shape}")

📥 Carregando movies_metadata.csv...
✅ 45,466 filmes carregados
📊 Dimensões originais: (45466, 24)


In [5]:
# Criar DataFrame Silver
movies_silver = movies_raw.copy()

print("🔧 Iniciando transformações...\n")

# 1. Converter tipos numéricos
print("1️⃣ Convertendo colunas numéricas...")
numeric_cols = ['budget', 'revenue', 'runtime', 'vote_average', 'vote_count', 'popularity']
for col in numeric_cols:
    movies_silver[col] = pd.to_numeric(movies_silver[col], errors='coerce')
print("   ✅ Colunas numéricas convertidas\n")

# 2. Converter data de lançamento
print("2️⃣ Convertendo datas...")
movies_silver['release_date'] = pd.to_datetime(movies_silver['release_date'], errors='coerce')
movies_silver['release_year'] = movies_silver['release_date'].dt.year
movies_silver['release_month'] = movies_silver['release_date'].dt.month
movies_silver['release_decade'] = (movies_silver['release_year'] // 10 * 10).astype('Int64')
print("   ✅ Datas convertidas e colunas derivadas criadas\n")

# 3. Criar colunas derivadas financeiras
print("3️⃣ Criando métricas financeiras...")
movies_silver['profit'] = movies_silver['revenue'] - movies_silver['budget']
movies_silver['roi'] = np.where(
    movies_silver['budget'] > 0,
    ((movies_silver['revenue'] - movies_silver['budget']) / movies_silver['budget'] * 100).round(2),
    None
)
print("   ✅ Profit e ROI calculados\n")

# 4. Parsing de campos JSON
print("4️⃣ Extraindo informações de campos JSON...")
movies_silver['genres_list'] = movies_silver['genres'].apply(extract_genres)
movies_silver['primary_genre'] = movies_silver['genres'].apply(lambda x: extract_first_name(x, 'name'))
movies_silver['production_companies_list'] = movies_silver['production_companies'].apply(extract_genres)
movies_silver['primary_company'] = movies_silver['production_companies'].apply(lambda x: extract_first_name(x, 'name'))
movies_silver['production_countries_list'] = movies_silver['production_countries'].apply(
    lambda x: extract_genres(x) if pd.notna(x) else None
)
movies_silver['primary_country'] = movies_silver['production_countries'].apply(
    lambda x: extract_first_name(x, 'name')
)
print("   ✅ Gêneros, produtoras e países extraídos\n")

# 5. Categorização de filmes
print("5️⃣ Categorizando filmes...")
# Categorias de orçamento
movies_silver['budget_category'] = pd.cut(
    movies_silver['budget'],
    bins=[0, 1000000, 10000000, 50000000, 100000000, float('inf')],
    labels=['Micro', 'Baixo', 'Médio', 'Alto', 'Blockbuster'],
    include_lowest=True
)

# Categorias de receita
movies_silver['revenue_category'] = pd.cut(
    movies_silver['revenue'],
    bins=[0, 1000000, 10000000, 100000000, 500000000, float('inf')],
    labels=['Muito Baixa', 'Baixa', 'Média', 'Alta', 'Excepcional'],
    include_lowest=True
)

# Categorias de duração
movies_silver['runtime_category'] = pd.cut(
    movies_silver['runtime'],
    bins=[0, 90, 120, 150, float('inf')],
    labels=['Curto', 'Médio', 'Longo', 'Muito Longo'],
    include_lowest=True
)
print("   ✅ Categorias de orçamento, receita e duração criadas\n")

# 6. Remover duplicatas
print("6️⃣ Removendo duplicatas...")
before = len(movies_silver)
movies_silver = movies_silver.drop_duplicates(subset=['id'])
after = len(movies_silver)
print(f"   ✅ {before - after} duplicatas removidas\n")

# 7. Selecionar e ordenar colunas
print("7️⃣ Organizando colunas...")
selected_columns = [
    'id', 'title', 'original_title', 'original_language',
    'release_date', 'release_year', 'release_month', 'release_decade',
    'budget', 'revenue', 'profit', 'roi',
    'budget_category', 'revenue_category',
    'runtime', 'runtime_category',
    'vote_average', 'vote_count', 'popularity',
    'genres_list', 'primary_genre',
    'production_companies_list', 'primary_company',
    'production_countries_list', 'primary_country',
    'status', 'adult',
    'overview', 'tagline', 'homepage',
    'imdb_id', 'poster_path'
]
movies_silver = movies_silver[selected_columns]
print(f"   ✅ {len(selected_columns)} colunas selecionadas\n")

print("\n" + "="*80)
print("📊 RESULTADO - MOVIES SILVER")
print("="*80)
print(f"Total de filmes: {len(movies_silver):,}")
print(f"Colunas: {movies_silver.shape[1]}")
print(f"\nPrimeiras linhas:")
movies_silver.head(3)

🔧 Iniciando transformações...

1️⃣ Convertendo colunas numéricas...
   ✅ Colunas numéricas convertidas

2️⃣ Convertendo datas...
   ✅ Datas convertidas e colunas derivadas criadas

3️⃣ Criando métricas financeiras...
   ✅ Profit e ROI calculados

4️⃣ Extraindo informações de campos JSON...
   ✅ Gêneros, produtoras e países extraídos

5️⃣ Categorizando filmes...
   ✅ Categorias de orçamento, receita e duração criadas

6️⃣ Removendo duplicatas...
   ✅ 30 duplicatas removidas

7️⃣ Organizando colunas...
   ✅ 32 colunas selecionadas


📊 RESULTADO - MOVIES SILVER
Total de filmes: 45,436
Colunas: 32

Primeiras linhas:


,id,title,original_title,original_language,release_date,release_year,release_month,release_decade,budget,revenue,profit,roi,budget_category,revenue_category,runtime,runtime_category,vote_average,vote_count,popularity,genres_list,primary_genre,production_companies_list,primary_company,production_countries_list,primary_country,status,adult,overview,tagline,homepage,imdb_id,poster_path
0,862,Toy Story,Toy Story,en,1995-10-30,1995.0,10.0,1990,30000000.0,373554033.0,343554033.0,1145.18,Médio,Alta,81.0,Curto,7.7,5415.0,21.946943,Animation|Comedy|Family,Animation,Pixar Animation Studios,Pixar Animation Studios,United States of America,United States of America,Released,False,"Led by Woody, Andy's toys live happily in his ...",NaN,http://toystory.disney.com/toy-story,tt0114709,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Jumanji,en,1995-12-15,1995.0,12.0,1990,65000000.0,262797249.0,197797249.0,304.3,Alto,Alta,104.0,Médio,6.9,2413.0,17.015539,Adventure|Fantasy|Family,Adventure,TriStar Pictures|Teitler Film|Interscope Commu...,TriStar Pictures,United States of America,United States of America,Released,False,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,NaN,tt0113497,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Grumpier Old Men,en,1995-12-22,1995.0,12.0,1990,0.0,0.0,0.0,None,Micro,Muito Baixa,101.0,Médio,6.5,92.0,11.712900,Romance|Comedy,Romance,Warner Bros.|Lancaster Gate,Warner Bros.,United States of America,United States of America,Released,False,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,NaN,tt0113228,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg


---
## 2️⃣ Transformação: Credits

In [6]:
# Carregar dados RAW
print("📥 Carregando credits.csv...")
credits_raw = pd.read_csv(raw_path / 'credits.csv')
print(f"✅ {len(credits_raw):,} registros carregados")

📥 Carregando credits.csv...
✅ 45,476 registros carregados


In [7]:
# Criar DataFrame Silver
credits_silver = credits_raw.copy()

print("🔧 Iniciando transformações...\n")

# 1. Extrair diretor
print("1️⃣ Extraindo diretor...")
credits_silver['director'] = credits_silver['crew'].apply(extract_director)
print(f"   ✅ Diretor extraído para {credits_silver['director'].notna().sum():,} filmes\n")

# 2. Extrair atores principais
print("2️⃣ Extraindo top 5 atores...")
credits_silver['top_actors'] = credits_silver['cast'].apply(lambda x: extract_top_actors(x, n=5))
credits_silver['lead_actor'] = credits_silver['cast'].apply(lambda x: extract_top_actors(x, n=1))
print(f"   ✅ Atores extraídos\n")

# 3. Contar membros do elenco e equipe
print("3️⃣ Contando membros...")
credits_silver['cast_size'] = credits_silver['cast'].apply(lambda x: len(safe_json_loads(x, [])))
credits_silver['crew_size'] = credits_silver['crew'].apply(lambda x: len(safe_json_loads(x, [])))
print(f"   ✅ Contagens realizadas\n")

# 4. Selecionar colunas
print("4️⃣ Selecionando colunas...")
credits_silver = credits_silver[[
    'id', 'director', 'lead_actor', 'top_actors', 'cast_size', 'crew_size'
]]
print(f"   ✅ Colunas selecionadas\n")

# 5. Remover duplicatas
print("5️⃣ Removendo duplicatas...")
before = len(credits_silver)
credits_silver = credits_silver.drop_duplicates(subset=['id'])
after = len(credits_silver)
print(f"   ✅ {before - after} duplicatas removidas\n")

print("\n" + "="*80)
print("📊 RESULTADO - CREDITS SILVER")
print("="*80)
print(f"Total de registros: {len(credits_silver):,}")
print(f"Filmes com diretor: {credits_silver['director'].notna().sum():,}")
print(f"Filmes com atores: {credits_silver['lead_actor'].notna().sum():,}")
print(f"\nPrimeiras linhas:")
credits_silver.head(3)

🔧 Iniciando transformações...

1️⃣ Extraindo diretor...
   ✅ Diretor extraído para 44,589 filmes

2️⃣ Extraindo top 5 atores...
   ✅ Atores extraídos

3️⃣ Contando membros...
   ✅ Contagens realizadas

4️⃣ Selecionando colunas...
   ✅ Colunas selecionadas

5️⃣ Removendo duplicatas...
   ✅ 44 duplicatas removidas


📊 RESULTADO - CREDITS SILVER
Total de registros: 45,432
Filmes com diretor: 44,545
Filmes com atores: 43,018

Primeiras linhas:


,id,director,lead_actor,top_actors,cast_size,crew_size
0,862,John Lasseter,Tom Hanks,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,13,106
1,8844,Joe Johnston,Robin Williams,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,26,16
2,15602,Howard Deutch,Walter Matthau,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,7,4


---
## 3️⃣ Transformação: Ratings

In [8]:
# Carregar dados RAW
print("📥 Carregando ratings.csv...")
ratings_raw = pd.read_csv(raw_path / 'ratings.csv')
print(f"✅ {len(ratings_raw):,} avaliações carregadas")

📥 Carregando ratings.csv...
✅ 26,024,289 avaliações carregadas


In [9]:
# Criar DataFrame Silver com agregações
print("🔧 Iniciando transformações...\n")

# 1. Converter timestamp
print("1️⃣ Convertendo timestamp...")
ratings_raw['rating_date'] = pd.to_datetime(ratings_raw['timestamp'], unit='s')
ratings_raw['rating_year'] = ratings_raw['rating_date'].dt.year
print("   ✅ Timestamps convertidos\n")

# 2. Agregar por filme
print("2️⃣ Agregando estatísticas por filme...")
ratings_agg = ratings_raw.groupby('movieId').agg({
    'rating': ['mean', 'median', 'std', 'count', 'min', 'max'],
    'userId': 'nunique'
}).round(2)

# Achatar colunas multi-índice
ratings_agg.columns = ['_'.join(col).strip() for col in ratings_agg.columns.values]
ratings_agg = ratings_agg.rename(columns={
    'rating_mean': 'avg_rating',
    'rating_median': 'median_rating',
    'rating_std': 'std_rating',
    'rating_count': 'total_ratings',
    'rating_min': 'min_rating',
    'rating_max': 'max_rating',
    'userId_nunique': 'unique_users'
})
ratings_agg = ratings_agg.reset_index()
print(f"   ✅ {len(ratings_agg):,} filmes com estatísticas agregadas\n")

# 3. Manter também dados detalhados (opcional - pode ser grande)
print("3️⃣ Preparando dados detalhados...")
ratings_detailed = ratings_raw[['userId', 'movieId', 'rating', 'rating_date', 'rating_year']].copy()
print(f"   ✅ {len(ratings_detailed):,} avaliações detalhadas preparadas\n")

print("\n" + "="*80)
print("📊 RESULTADO - RATINGS SILVER")
print("="*80)
print(f"Filmes com avaliações agregadas: {len(ratings_agg):,}")
print(f"Total de avaliações detalhadas: {len(ratings_detailed):,}")
print(f"\nEstatísticas agregadas - Primeiras linhas:")
ratings_agg.head(3)

🔧 Iniciando transformações...

1️⃣ Convertendo timestamp...
   ✅ Timestamps convertidos

2️⃣ Agregando estatísticas por filme...
   ✅ 45,115 filmes com estatísticas agregadas

3️⃣ Preparando dados detalhados...
   ✅ 26,024,289 avaliações detalhadas preparadas


📊 RESULTADO - RATINGS SILVER
Filmes com avaliações agregadas: 45,115
Total de avaliações detalhadas: 26,024,289

Estatísticas agregadas - Primeiras linhas:


,movieId,avg_rating,median_rating,std_rating,total_ratings,min_rating,max_rating,unique_users
0,1,3.89,4.0,0.93,66008,0.5,5.0,66008
1,2,3.24,3.0,0.96,26060,0.5,5.0,26060
2,3,3.18,3.0,1.01,15497,0.5,5.0,15497


---
## 4️⃣ Transformação: Keywords

In [10]:
# Carregar dados RAW
print("📥 Carregando keywords.csv...")
keywords_raw = pd.read_csv(raw_path / 'keywords.csv')
print(f"✅ {len(keywords_raw):,} registros carregados")

📥 Carregando keywords.csv...
✅ 46,419 registros carregados


In [11]:
# Criar DataFrame Silver
keywords_silver = keywords_raw.copy()

print("🔧 Iniciando transformações...\n")

# 1. Extrair keywords como string
print("1️⃣ Extraindo keywords...")
keywords_silver['keywords_list'] = keywords_silver['keywords'].apply(extract_genres)
keywords_silver['keywords_count'] = keywords_silver['keywords'].apply(
    lambda x: len(safe_json_loads(x, []))
)
print(f"   ✅ Keywords extraídas\n")

# 2. Selecionar colunas
print("2️⃣ Selecionando colunas...")
keywords_silver = keywords_silver[['id', 'keywords_list', 'keywords_count']]
print(f"   ✅ Colunas selecionadas\n")

# 3. Remover duplicatas
print("3️⃣ Removendo duplicatas...")
before = len(keywords_silver)
keywords_silver = keywords_silver.drop_duplicates(subset=['id'])
after = len(keywords_silver)
print(f"   ✅ {before - after} duplicatas removidas\n")

print("\n" + "="*80)
print("📊 RESULTADO - KEYWORDS SILVER")
print("="*80)
print(f"Total de registros: {len(keywords_silver):,}")
print(f"Filmes com keywords: {keywords_silver['keywords_list'].notna().sum():,}")
print(f"\nPrimeiras linhas:")
keywords_silver.head(3)

🔧 Iniciando transformações...

1️⃣ Extraindo keywords...
   ✅ Keywords extraídas

2️⃣ Selecionando colunas...
   ✅ Colunas selecionadas

3️⃣ Removendo duplicatas...
   ✅ 987 duplicatas removidas


📊 RESULTADO - KEYWORDS SILVER
Total de registros: 45,432
Filmes com keywords: 31,092

Primeiras linhas:


,id,keywords_list,keywords_count
0,862,jealousy|toy|boy|friendship|friends|rivalry|bo...,9
1,8844,board game|disappearance|based on children's b...,6
2,15602,fishing|best friend|duringcreditsstinger|old men,4


---
## 5️⃣ Transformação: Links

In [12]:
# Carregar dados RAW
print("📥 Carregando links.csv...")
links_raw = pd.read_csv(raw_path / 'links.csv')
print(f"✅ {len(links_raw):,} registros carregados")

📥 Carregando links.csv...
✅ 45,843 registros carregados


In [13]:
# Criar DataFrame Silver
links_silver = links_raw.copy()

print("🔧 Iniciando transformações...\n")

# 1. Converter tipos
print("1️⃣ Convertendo tipos de dados...")
links_silver['tmdbId'] = pd.to_numeric(links_silver['tmdbId'], errors='coerce').astype('Int64')
links_silver['imdbId'] = links_silver['imdbId'].astype(str).str.zfill(7)
links_silver['imdbId_formatted'] = 'tt' + links_silver['imdbId']
print("   ✅ Tipos convertidos e IMDB ID formatado\n")

# 2. Remover registros sem tmdbId (não podem ser linkados)
print("2️⃣ Removendo registros sem tmdbId...")
before = len(links_silver)
links_silver = links_silver.dropna(subset=['tmdbId'])
after = len(links_silver)
print(f"   ✅ {before - after} registros sem tmdbId removidos\n")

# 3. Remover duplicatas
print("3️⃣ Removendo duplicatas...")
before = len(links_silver)
links_silver = links_silver.drop_duplicates(subset=['movieId'])
after = len(links_silver)
print(f"   ✅ {before - after} duplicatas removidas\n")

print("\n" + "="*80)
print("📊 RESULTADO - LINKS SILVER")
print("="*80)
print(f"Total de registros: {len(links_silver):,}")
print(f"\nPrimeiras linhas:")
links_silver.head(3)

🔧 Iniciando transformações...

1️⃣ Convertendo tipos de dados...
   ✅ Tipos convertidos e IMDB ID formatado

2️⃣ Removendo registros sem tmdbId...
   ✅ 219 registros sem tmdbId removidos

3️⃣ Removendo duplicatas...
   ✅ 0 duplicatas removidas


📊 RESULTADO - LINKS SILVER
Total de registros: 45,624

Primeiras linhas:


,movieId,imdbId,tmdbId,imdbId_formatted
0,1,0114709,862,tt0114709
1,2,0113497,8844,tt0113497
2,3,0113228,15602,tt0113228


---
## 6️⃣ Integração e Validação

In [16]:
# Merge movies com credits
print("🔗 Integrando dados...\n")

# Padronizar tipos de ID em todos os dataframes
print("0️⃣ Padronizando tipos de ID...")
movies_silver['id'] = pd.to_numeric(movies_silver['id'], errors='coerce').astype('Int64')
credits_silver['id'] = pd.to_numeric(credits_silver['id'], errors='coerce').astype('Int64')
keywords_silver['id'] = pd.to_numeric(keywords_silver['id'], errors='coerce').astype('Int64')
print("   ✅ IDs convertidos para Int64\n")

print("1️⃣ Merge: Movies + Credits...")
movies_full = movies_silver.merge(credits_silver, on='id', how='left')
print(f"   ✅ {len(movies_full):,} filmes após merge")
print(f"   📊 Filmes com diretor: {movies_full['director'].notna().sum():,}\n")

print("2️⃣ Merge: Movies + Keywords...")
movies_full = movies_full.merge(keywords_silver, on='id', how='left')
print(f"   ✅ {len(movies_full):,} filmes após merge")
print(f"   📊 Filmes com keywords: {movies_full['keywords_list'].notna().sum():,}\n")

print("3️⃣ Merge: Movies + Ratings (via Links)...")
# Primeiro merge links com ratings agregados
links_ratings = links_silver.merge(ratings_agg, on='movieId', how='left')
# Depois merge com movies usando tmdbId
movies_full = movies_full.merge(
    links_ratings[['tmdbId', 'avg_rating', 'median_rating', 'total_ratings', 'unique_users']],
    left_on='id',
    right_on='tmdbId',
    how='left'
)
movies_full = movies_full.drop('tmdbId', axis=1)
print(f"   ✅ {len(movies_full):,} filmes após merge")
print(f"   📊 Filmes com ratings do usuário: {movies_full['avg_rating'].notna().sum():,}\n")

print("\n" + "="*80)
print("📊 DATASET INTEGRADO - MOVIES FULL")
print("="*80)
print(f"Total de filmes: {len(movies_full):,}")
print(f"Total de colunas: {movies_full.shape[1]}")
print(f"\nColunas disponíveis:")
print(movies_full.columns.tolist())

🔗 Integrando dados...

0️⃣ Padronizando tipos de ID...
   ✅ IDs convertidos para Int64

1️⃣ Merge: Movies + Credits...
   ✅ 45,436 filmes após merge
   📊 Filmes com diretor: 44,545

2️⃣ Merge: Movies + Keywords...
   ✅ 45,436 filmes após merge
   📊 Filmes com keywords: 31,092

3️⃣ Merge: Movies + Ratings (via Links)...
   ✅ 45,466 filmes após merge
   📊 Filmes com ratings do usuário: 44,741


📊 DATASET INTEGRADO - MOVIES FULL
Total de filmes: 45,466
Total de colunas: 43

Colunas disponíveis:
['id', 'title', 'original_title', 'original_language', 'release_date', 'release_year', 'release_month', 'release_decade', 'budget', 'revenue', 'profit', 'roi', 'budget_category', 'revenue_category', 'runtime', 'runtime_category', 'vote_average', 'vote_count', 'popularity', 'genres_list', 'primary_genre', 'production_companies_list', 'primary_company', 'production_countries_list', 'primary_country', 'status', 'adult', 'overview', 'tagline', 'homepage', 'imdb_id', 'poster_path', 'director', 'lead_act

In [17]:
# Estatísticas de qualidade
print("\n" + "="*80)
print("📈 ESTATÍSTICAS DE QUALIDADE - CAMADA SILVER")
print("="*80 + "\n")

print("COMPLETUDE DOS DADOS:")
print("-" * 80)
completeness = pd.DataFrame({
    'Coluna': ['budget', 'revenue', 'runtime', 'director', 'genres_list', 
               'avg_rating', 'keywords_list'],
    'Não-Nulos': [
        (movies_full['budget'] > 0).sum(),
        (movies_full['revenue'] > 0).sum(),
        movies_full['runtime'].notna().sum(),
        movies_full['director'].notna().sum(),
        movies_full['genres_list'].notna().sum(),
        movies_full['avg_rating'].notna().sum(),
        movies_full['keywords_list'].notna().sum()
    ]
})
completeness['Percentual'] = (completeness['Não-Nulos'] / len(movies_full) * 100).round(2)
print(completeness.to_string(index=False))

print("\n" + "="*80)
print("✅ Transformação concluída com sucesso!")


📈 ESTATÍSTICAS DE QUALIDADE - CAMADA SILVER

COMPLETUDE DOS DADOS:
--------------------------------------------------------------------------------
       Coluna  Não-Nulos  Percentual
       budget       8890       19.55
      revenue       7408       16.29
      runtime      45203       99.42
     director      44575       98.04
  genres_list      43024       94.63
   avg_rating      44741       98.41
keywords_list      31110       68.42

✅ Transformação concluída com sucesso!


---
## 7️⃣ Salvar Dados na Camada SILVER

In [18]:
print("💾 Salvando dados transformados...\n")

# Salvar dataset integrado principal
print("1️⃣ Salvando movies_full.csv...")
movies_full.to_csv(silver_path / 'movies_full.csv', index=False)
print(f"   ✅ {len(movies_full):,} filmes salvos\n")

# Salvar datasets individuais
print("2️⃣ Salvando movies.csv...")
movies_silver.to_csv(silver_path / 'movies.csv', index=False)
print(f"   ✅ {len(movies_silver):,} filmes salvos\n")

print("3️⃣ Salvando credits.csv...")
credits_silver.to_csv(silver_path / 'credits.csv', index=False)
print(f"   ✅ {len(credits_silver):,} registros salvos\n")

print("4️⃣ Salvando ratings_aggregated.csv...")
ratings_agg.to_csv(silver_path / 'ratings_aggregated.csv', index=False)
print(f"   ✅ {len(ratings_agg):,} filmes com estatísticas salvos\n")

print("5️⃣ Salvando keywords.csv...")
keywords_silver.to_csv(silver_path / 'keywords.csv', index=False)
print(f"   ✅ {len(keywords_silver):,} registros salvos\n")

print("6️⃣ Salvando links.csv...")
links_silver.to_csv(silver_path / 'links.csv', index=False)
print(f"   ✅ {len(links_silver):,} registros salvos\n")

# Nota: ratings_detailed é muito grande, salvar apenas se necessário
print("⚠️  ratings_detailed não foi salvo (volume muito grande)")
print("   Se necessário, descomente a linha abaixo:")
print("   # ratings_detailed.to_csv(silver_path / 'ratings_detailed.csv', index=False)")

print("\n" + "="*80)
print("✅ TODOS OS DADOS SALVOS NA CAMADA SILVER!")
print("="*80)
print(f"📂 Localização: {silver_path}")
print(f"📅 Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

💾 Salvando dados transformados...

1️⃣ Salvando movies_full.csv...
   ✅ 45,466 filmes salvos

2️⃣ Salvando movies.csv...
   ✅ 45,436 filmes salvos

3️⃣ Salvando credits.csv...
   ✅ 45,432 registros salvos

4️⃣ Salvando ratings_aggregated.csv...
   ✅ 45,115 filmes com estatísticas salvos

5️⃣ Salvando keywords.csv...
   ✅ 45,432 registros salvos

6️⃣ Salvando links.csv...
   ✅ 45,624 registros salvos

⚠️  ratings_detailed não foi salvo (volume muito grande)
   Se necessário, descomente a linha abaixo:
   # ratings_detailed.to_csv(silver_path / 'ratings_detailed.csv', index=False)

✅ TODOS OS DADOS SALVOS NA CAMADA SILVER!
📂 Localização: c:\Users\Lenovo\Documents\Pedro\Bancos2\projeto-bancos-dados-2\Data Layer\silver\dados_processados
📅 Data/Hora: 2025-11-23 16:19:26


---
## 📋 Resumo da Transformação

### ✅ Dados Transformados:

1. **movies_full.csv** - Dataset integrado principal
   - Filmes + Credits + Keywords + Ratings
   - Pronto para análises complexas

2. **movies.csv** - Filmes transformados
   - Colunas numéricas convertidas
   - Datas processadas
   - Métricas derivadas (profit, ROI)
   - Categorias criadas
   - JSON parseado (genres, companies, countries)

3. **credits.csv** - Créditos processados
   - Diretor extraído
   - Top 5 atores principais
   - Contagens de elenco/equipe

4. **ratings_aggregated.csv** - Estatísticas de avaliações
   - Média, mediana, desvio padrão
   - Total de avaliações
   - Usuários únicos

5. **keywords.csv** - Keywords processadas
   - Keywords extraídas como string
   - Contagem de keywords

6. **links.csv** - Links padronizados
   - IDs formatados
   - Registros válidos

### 🎯 Próximo Passo: Camada GOLD
- Criar modelo dimensional (Star Schema)
- Definir tabela fato e dimensões
- Preparar para Power BI

---
## 8️⃣ Carregar no PostgreSQL (Schema SILVER)

Agora vamos consolidar todos os dados e carregar na tabela única `silver.movies_raw` conforme requisito do chefe.

In [20]:
# Consolidar dados em um único DataFrame para silver.movies_raw
print("🔗 Consolidando dados para tabela única silver.movies_raw...\n")

# Começar com movies_full que já tem movies + credits + keywords + ratings parciais
df_consolidated = movies_full.copy()

# Adicionar colunas de links (tmdbId e imdbId formatado)
print("1️⃣ Adicionando informações de links...")
links_info = links_silver[['movieId', 'tmdbId', 'imdbId_formatted']].copy()
links_info = links_info.rename(columns={'movieId': 'id'})

# Fazer merge usando tmdbId que é o mesmo que id em movies
df_consolidated = df_consolidated.merge(
    links_info[['tmdbId', 'imdbId_formatted']], 
    left_on='id', 
    right_on='tmdbId', 
    how='left'
)
df_consolidated = df_consolidated.drop('tmdbId', axis=1)
df_consolidated = df_consolidated.rename(columns={'imdbId_formatted': 'imdb_id_formatted'})
print(f"   ✅ Links adicionados\n")

# Adicionar colunas faltantes de ratings (std_rating, min_rating, max_rating)
print("2️⃣ Adicionando estatísticas completas de ratings...")
# Fazer merge com links_ratings para pegar TODAS as colunas de ratings
links_ratings_full = links_silver[['tmdbId', 'movieId']].merge(
    ratings_agg, 
    on='movieId', 
    how='left'
)

# Merge com df_consolidated para adicionar colunas faltantes
df_consolidated = df_consolidated.merge(
    links_ratings_full[['tmdbId', 'std_rating', 'min_rating', 'max_rating']],
    left_on='id',
    right_on='tmdbId',
    how='left',
    suffixes=('', '_new')
)
df_consolidated = df_consolidated.drop('tmdbId', axis=1)
print(f"   ✅ Estatísticas completas de ratings adicionadas\n")

# Reorganizar colunas para corresponder ao schema silver.movies_raw
print("3️⃣ Organizando colunas conforme schema silver.movies_raw...")
final_columns = [
    'id', 'title', 'original_title', 'original_language',
    'release_date', 'release_year', 'release_month', 'release_decade',
    'budget', 'revenue', 'profit', 'roi',
    'budget_category', 'revenue_category',
    'runtime', 'runtime_category',
    'vote_average', 'vote_count', 'popularity',
    'genres_list', 'primary_genre',
    'production_companies_list', 'primary_company',
    'production_countries_list', 'primary_country',
    'status', 'adult',
    'overview', 'tagline', 'homepage',
    'imdb_id', 'poster_path',
    'director', 'lead_actor', 'top_actors', 'cast_size', 'crew_size',
    'keywords_list', 'keywords_count',
    'avg_rating', 'median_rating', 'std_rating', 'total_ratings', 
    'min_rating', 'max_rating', 'unique_users',
    'tmdb_id', 'imdb_id_formatted'
]

# Adicionar coluna tmdb_id que é igual a id
df_consolidated['tmdb_id'] = df_consolidated['id']

# Selecionar apenas as colunas necessárias
df_consolidated = df_consolidated[final_columns]
print(f"   ✅ {len(final_columns)} colunas organizadas\n")

# Limpar valores None convertendo para None explícito
print("4️⃣ Limpando valores nulos...")
df_consolidated = df_consolidated.where(pd.notnull(df_consolidated), None)
print(f"   ✅ Valores None padronizados\n")

print("\n" + "="*80)
print("📊 DATASET CONSOLIDADO FINAL")
print("="*80)
print(f"Total de filmes: {len(df_consolidated):,}")
print(f"Total de colunas: {len(df_consolidated.columns)}")
print(f"\nPrimeiras linhas:")
df_consolidated.head(3)

🔗 Consolidando dados para tabela única silver.movies_raw...

1️⃣ Adicionando informações de links...
   ✅ Links adicionados

2️⃣ Adicionando estatísticas completas de ratings...
   ✅ Estatísticas completas de ratings adicionadas

3️⃣ Organizando colunas conforme schema silver.movies_raw...
   ✅ 48 colunas organizadas

4️⃣ Limpando valores nulos...
   ✅ Valores None padronizados


📊 DATASET CONSOLIDADO FINAL
Total de filmes: 45,658
Total de colunas: 48

Primeiras linhas:
   ✅ Estatísticas completas de ratings adicionadas

3️⃣ Organizando colunas conforme schema silver.movies_raw...
   ✅ 48 colunas organizadas

4️⃣ Limpando valores nulos...
   ✅ Valores None padronizados


📊 DATASET CONSOLIDADO FINAL
Total de filmes: 45,658
Total de colunas: 48

Primeiras linhas:


,id,title,original_title,original_language,release_date,release_year,release_month,release_decade,budget,revenue,profit,roi,budget_category,revenue_category,runtime,runtime_category,vote_average,vote_count,popularity,genres_list,primary_genre,production_companies_list,primary_company,production_countries_list,primary_country,status,adult,overview,tagline,homepage,imdb_id,poster_path,director,lead_actor,top_actors,cast_size,crew_size,keywords_list,keywords_count,avg_rating,median_rating,std_rating,total_ratings,min_rating,max_rating,unique_users,tmdb_id,imdb_id_formatted
0,862,Toy Story,Toy Story,en,1995-10-30,1995.0,10.0,1990,30000000.0,373554033.0,343554033.0,1145.18,Médio,Alta,81.0,Curto,7.7,5415.0,21.946943,Animation|Comedy|Family,Animation,Pixar Animation Studios,Pixar Animation Studios,United States of America,United States of America,Released,False,"Led by Woody, Andy's toys live happily in his ...",None,http://toystory.disney.com/toy-story,tt0114709,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,John Lasseter,Tom Hanks,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,13.0,106.0,jealousy|toy|boy|friendship|friends|rivalry|bo...,9.0,3.89,4.0,0.93,66008.0,0.5,5.0,66008.0,862,tt0114709
1,8844,Jumanji,Jumanji,en,1995-12-15,1995.0,12.0,1990,65000000.0,262797249.0,197797249.0,304.3,Alto,Alta,104.0,Médio,6.9,2413.0,17.015539,Adventure|Fantasy|Family,Adventure,TriStar Pictures|Teitler Film|Interscope Commu...,TriStar Pictures,United States of America,United States of America,Released,False,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!,None,tt0113497,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Joe Johnston,Robin Williams,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,26.0,16.0,board game|disappearance|based on children's b...,6.0,3.24,3.0,0.96,26060.0,0.5,5.0,26060.0,8844,tt0113497
2,15602,Grumpier Old Men,Grumpier Old Men,en,1995-12-22,1995.0,12.0,1990,0.0,0.0,0.0,None,Micro,Muito Baixa,101.0,Médio,6.5,92.0,11.712900,Romance|Comedy,Romance,Warner Bros.|Lancaster Gate,Warner Bros.,United States of America,United States of America,Released,False,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...,None,tt0113228,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Howard Deutch,Walter Matthau,Walter Matthau|Jack Lemmon|Ann-Margret|Sophia ...,7.0,4.0,fishing|best friend|duringcreditsstinger|old men,4.0,3.18,3.0,1.01,15497.0,0.5,5.0,15497.0,15602,tt0113228


In [22]:
import psycopg2
from psycopg2.extras import execute_values

# Configurações de conexão (conforme docker-compose.yml)
DB_CONFIG = {
    'dbname': 'movies_dw',
    'user': 'postgres',
    'password': 'postgres123',  # Senha correta do docker-compose.yml
    'host': 'localhost',
    'port': '5432'
}

print("🔌 Conectando ao PostgreSQL...")
try:
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    print("   ✅ Conexão estabelecida com sucesso!\n")
    
    # Verificar se tabela existe
    cursor.execute("""
        SELECT EXISTS (
            SELECT FROM information_schema.tables 
            WHERE table_schema = 'silver' 
            AND table_name = 'movies_raw'
        );
    """)
    table_exists = cursor.fetchone()[0]
    
    if table_exists:
        print("📋 Tabela silver.movies_raw encontrada!")
        
        # Contar registros atuais
        cursor.execute("SELECT COUNT(*) FROM silver.movies_raw;")
        current_count = cursor.fetchone()[0]
        print(f"   Registros atuais: {current_count:,}\n")
    else:
        print("⚠️  Tabela silver.movies_raw não encontrada!")
        print("   Execute o script DDL primeiro.\n")
        
except Exception as e:
    print(f"❌ Erro na conexão: {e}")
    raise

🔌 Conectando ao PostgreSQL...
   ✅ Conexão estabelecida com sucesso!

📋 Tabela silver.movies_raw encontrada!
   Registros atuais: 0



In [25]:
print("🗑️  Limpando tabela silver.movies_raw...")
try:
    cursor.execute("TRUNCATE TABLE silver.movies_raw;")
    conn.commit()
    print("   ✅ Tabela limpa com sucesso!\n")
except Exception as e:
    print(f"❌ Erro ao limpar tabela: {e}")
    conn.rollback()
    raise

# Verificar e remover registros com ID nulo
print("🔍 Verificando integridade dos dados...")
null_ids = df_consolidated['id'].isna().sum()
if null_ids > 0:
    print(f"   ⚠️  Encontrados {null_ids} registros com ID nulo!")
    print(f"   🧹 Removendo registros sem ID...")
    df_clean = df_consolidated.dropna(subset=['id']).copy()
    print(f"   ✅ Registros com ID nulo removidos\n")
else:
    print(f"   ✅ Todos os registros têm ID válido\n")
    df_clean = df_consolidated.copy()

# Verificar e remover duplicatas
print("🔍 Verificando duplicatas no DataFrame...")
duplicates_before = df_clean.duplicated(subset=['id']).sum()
if duplicates_before > 0:
    print(f"   ⚠️  Encontradas {duplicates_before} linhas duplicadas!")
    print(f"   🧹 Removendo duplicatas mantendo a primeira ocorrência...")
    df_clean = df_clean.drop_duplicates(subset=['id'], keep='first')
    print(f"   ✅ DataFrame limpo: {len(df_clean):,} filmes únicos\n")
else:
    print(f"   ✅ Nenhuma duplicata encontrada\n")

print(f"💾 Carregando {len(df_clean):,} filmes na tabela silver.movies_raw...")
print("   Isso pode levar alguns minutos...\n")

# Preparar dados para inserção
data_tuples = []
for idx, row in df_clean.iterrows():
    # Converter valores para tipos compatíveis com PostgreSQL
    row_data = tuple(
        None if pd.isna(val) else val 
        for val in row
    )
    data_tuples.append(row_data)
    
    # Mostrar progresso a cada 10000 registros
    if (idx + 1) % 10000 == 0:
        print(f"   📦 Preparado: {idx + 1:,} registros...")

print(f"   ✅ {len(data_tuples):,} registros preparados!\n")

# SQL de inserção
insert_sql = """
INSERT INTO silver.movies_raw (
    id, title, original_title, original_language,
    release_date, release_year, release_month, release_decade,
    budget, revenue, profit, roi,
    budget_category, revenue_category,
    runtime, runtime_category,
    vote_average, vote_count, popularity,
    genres_list, primary_genre,
    production_companies_list, primary_company,
    production_countries_list, primary_country,
    status, adult,
    overview, tagline, homepage,
    imdb_id, poster_path,
    director, lead_actor, top_actors, cast_size, crew_size,
    keywords_list, keywords_count,
    avg_rating, median_rating, std_rating, total_ratings, 
    min_rating, max_rating, unique_users,
    tmdb_id, imdb_id_formatted
) VALUES %s
"""

try:
    print("⏳ Executando inserção em lote...")
    execute_values(cursor, insert_sql, data_tuples, page_size=1000)
    conn.commit()
    
    # Verificar quantos registros foram inseridos
    cursor.execute("SELECT COUNT(*) FROM silver.movies_raw;")
    final_count = cursor.fetchone()[0]
    
    print(f"\n{'='*80}")
    print("✅ DADOS CARREGADOS COM SUCESSO!")
    print(f"{'='*80}")
    print(f"Total de filmes inseridos: {final_count:,}")
    print(f"Tempo de processamento: Concluído")
    print(f"{'='*80}\n")
    
except Exception as e:
    print(f"\n❌ Erro durante inserção: {e}")
    conn.rollback()
    raise

🗑️  Limpando tabela silver.movies_raw...
   ✅ Tabela limpa com sucesso!

🔍 Verificando integridade dos dados...
   ⚠️  Encontrados 3 registros com ID nulo!
   🧹 Removendo registros sem ID...
   ✅ Registros com ID nulo removidos

🔍 Verificando duplicatas no DataFrame...
   ⚠️  Encontradas 222 linhas duplicadas!
   🧹 Removendo duplicatas mantendo a primeira ocorrência...
   ✅ DataFrame limpo: 45,433 filmes únicos

💾 Carregando 45,433 filmes na tabela silver.movies_raw...
   Isso pode levar alguns minutos...

   📦 Preparado: 10,000 registros...
   📦 Preparado: 10,000 registros...
   📦 Preparado: 20,000 registros...
   📦 Preparado: 20,000 registros...
   📦 Preparado: 30,000 registros...
   📦 Preparado: 30,000 registros...
   📦 Preparado: 40,000 registros...
   📦 Preparado: 40,000 registros...
   ✅ 45,433 registros preparados!

⏳ Executando inserção em lote...
   ✅ 45,433 registros preparados!

⏳ Executando inserção em lote...

✅ DADOS CARREGADOS COM SUCESSO!
Total de filmes inseridos: 45,4

In [26]:
print("🔍 Validando dados carregados...\n")

# 1. Contagem por década
print("📊 Filmes por década:")
cursor.execute("""
    SELECT release_decade, COUNT(*) as total
    FROM silver.movies_raw
    WHERE release_decade IS NOT NULL
    GROUP BY release_decade
    ORDER BY release_decade DESC
    LIMIT 5;
""")
for row in cursor.fetchall():
    print(f"   {row[0]}: {row[1]:,} filmes")

print("\n📊 Top 5 diretores:")
cursor.execute("""
    SELECT director, COUNT(*) as total
    FROM silver.movies_raw
    WHERE director IS NOT NULL
    GROUP BY director
    ORDER BY total DESC
    LIMIT 5;
""")
for row in cursor.fetchall():
    print(f"   {row[0]}: {row[1]:,} filmes")

print("\n📊 Top 5 gêneros principais:")
cursor.execute("""
    SELECT primary_genre, COUNT(*) as total
    FROM silver.movies_raw
    WHERE primary_genre IS NOT NULL
    GROUP BY primary_genre
    ORDER BY total DESC
    LIMIT 5;
""")
for row in cursor.fetchall():
    print(f"   {row[0]}: {row[1]:,} filmes")

print("\n📊 Estatísticas de rating:")
cursor.execute("""
    SELECT 
        ROUND(AVG(avg_rating)::numeric, 2) as media_geral,
        ROUND(MIN(avg_rating)::numeric, 2) as min_rating,
        ROUND(MAX(avg_rating)::numeric, 2) as max_rating,
        COUNT(*) FILTER (WHERE avg_rating IS NOT NULL) as filmes_com_rating
    FROM silver.movies_raw;
""")
stats = cursor.fetchone()
print(f"   Média geral: {stats[0]}")
print(f"   Rating mínimo: {stats[1]}")
print(f"   Rating máximo: {stats[2]}")
print(f"   Filmes com rating: {stats[3]:,}")

print("\n" + "="*80)
print("✅ VALIDAÇÃO CONCLUÍDA - Dados estão corretos na tabela silver.movies_raw!")
print("="*80)

# Fechar conexão
cursor.close()
conn.close()
print("\n🔌 Conexão com PostgreSQL encerrada.")

🔍 Validando dados carregados...

📊 Filmes por década:
   2020: 1 filmes
   2010: 12,793 filmes
   2000: 11,195 filmes
   1990: 5,675 filmes
   1980: 3,928 filmes

📊 Top 5 diretores:
   John Ford: 66 filmes
   Michael Curtiz: 65 filmes
   Werner Herzog: 54 filmes
   Alfred Hitchcock: 53 filmes
   Georges Méliès: 51 filmes

📊 Top 5 gêneros principais:
   Drama: 11,953 filmes
   Comedy: 8,816 filmes
   Action: 4,487 filmes
   Documentary: 3,413 filmes
   Horror: 2,619 filmes

📊 Estatísticas de rating:
   Média geral: 3.06
   Rating mínimo: 0.50
   Rating máximo: 5.00
   Filmes com rating: 44,711

✅ VALIDAÇÃO CONCLUÍDA - Dados estão corretos na tabela silver.movies_raw!

🔌 Conexão com PostgreSQL encerrada.
